In [ ]:
# %load tp2_AStar_Sergiy_Goloviatinski.py
#!/usr/bin/env python3

# Sergiy Goloviatinski, inf3dlm-b


from city import *
from collections import deque
from math import sqrt
import heapq
import inspect
import sys

def initCitiesFromFile(fileName):
    cityDict = {}
    with open(fileName) as f:
        for c in [l.split() for l in f]:
            cityName = c[0]
            x = c[1]
            y = c[2]
            cityDict[cityName] = City(cityName, x, y)
    return cityDict


def initCityConnections(cityDict, connectionFileName):
    with open(connectionFileName) as f:
        for c in [l.split() for l in f]:
            distance = c[2]
            cityA = c[0]
            cityB = c[1]
            cityDict[cityA].addNeighbour(cityDict[cityB], int(distance))
            cityDict[cityB].addNeighbour(cityDict[cityA], int(distance))


def getCityStatesNames(collection):
    return [citystate.city.name for citystate in collection]

def getCityStateFromName(cityName,collection):
    for citystate in collection:
         if citystate.city.name==cityName:
             return citystate
    return None

# g(x) s'accumulle avec chaque noeud visité (somme des distances parcourues) et h(x) est donné dans le pdf (se réduit à chaque noeud visité vu qu'on se rapproche de la ville d'arrivée)
# Fortement inspiré de https://fr.wikipedia.org/wiki/Algorithme_A*
def findShortestPathFromAtoB(cityA, cityB, h, printPathInfo=True):
    cityState = CityState(cityA,0,h(cityA,cityB))
    closedList=deque()
    openList=[cityState]
    heapq.heapify(openList)

    searchCount=0

    while len(openList)>0:

        cityState=heapq.heappop(openList)

        if cityState.city == cityB:
            parent=cityState.parent
            path=[cityState]
            while parent is not None:
                path.append(parent)
                parent=parent.parent
            if printPathInfo:
                print(f"\tFound path (length = {len(path)}) from {cityA.name} to {cityB.name} after exploring {searchCount} states")
            return list(reversed(path))

        searchCount+=1
        neighbours = cityState.city.neighbours

        for neighbourCity, neighbourDistance in neighbours.items():
            neighbourCityState=CityState(neighbourCity,cityState.g+neighbourDistance,h(neighbourCity,cityB) ,cityState)
            name=neighbourCityState.city.name

            if name in getCityStatesNames(closedList) and neighbourCityState.f >getCityStateFromName(name,closedList).f or name in getCityStatesNames(openList) and neighbourCityState.f > getCityStateFromName(name,openList).f:
                pass
            else:
                heapq.heappush(openList,neighbourCityState)

        closedList.append(cityState)

cityDict = initCitiesFromFile("data/positions.txt")
initCityConnections(cityDict, "data/connections.txt")

heuristicList=[]

h0 = lambda a,b: 0
h1 = lambda a,b: abs(a.x-b.x)
h2 = lambda a,b: abs(a.y-b.y)
h3 = lambda a,b: sqrt(abs(a.x-b.x)**2 + abs(a.y-b.y)**2)
h4 = lambda a,b: abs(a.x-b.x) + abs(a.y-b.y)

heuristicList.extend([h0,h1,h2,h3,h4])

if __name__ == '__main__':

    cityTupleList=[]

    if len(sys.argv)>1 and (len(sys.argv)-1)%2==0:
        for i in range(1,len(sys.argv)-1,2):
            try:
                cityTupleList.append((cityDict[sys.argv[i].capitalize()],cityDict[sys.argv[i+1].capitalize()]))
            except KeyError as e:
                raise KeyError(f"The city you specified ({e.args[0]}) don't exist in our dataset")
    elif len(sys.argv)>1 and (len(sys.argv)-1)%2!=0:
        raise Exception("The cities must be given by pairs")
    else:
        # Default test values
        cityTupleList.append((cityDict["Bern"],cityDict["Amsterdam"]))
        cityTupleList.append((cityDict["Lisbon"],cityDict["Warsaw"]))


    for cityA, cityB in cityTupleList:
        print(f"Find shorthest path from {cityA.name} to {cityB.name} with {len(heuristicList)} different heuristics:\n")
        for h in heuristicList:
            print(f"\tHeuristic {inspect.getsource(h)}")
            spaces=[""]
            for state in findShortestPathFromAtoB(cityA, cityB, h):
                print(f"\t\t{''.join(spaces)}{'⮡ ' if len(spaces)>1 else ''}{state}")
                spaces.append(" ")
            print()
        print()


# Questions

## L’utilisation des différentes heuristiques a-t-elle une influence sur l’efficacité de la recherche ?

Oui, forcément car par exemple avec l'heuristique h0 ça revient à utiliser un parcours "breadth-first search" donc on visitera tous les noeuds à chaque fois avant de trouver l'optimum 

Plus l'estimation du coût par l'heuristique se rapproche du coût réel, moins de noeuds l'algorithme devra explorer pour trouver le chemin optimum

Dans les exemples plus loin dans ce document, on peut constater que le nombre de noeuds explorés avant de trouver le chemin optimum diffère à chaque fois entre les heuristiques

## Pouvez-vous trouver des exemples où l’utilisation de différentes heuristiques donne des résultats différents en termes de chemin trouvé ?

Vu que l'heuristique h4 qui utilise la distance de Manhattan n'est pas admissible (elle peut surestimer le coût), dans certains cas le chemin trouvé par l'algorithme ne sera pas le même qu'avec les autres heuristiques dans le cas où cette surestimation de coût influencera sur la décision de l'algorithme par quelle ville trouver le chemin le plus court.

Vu que les autres heuristiques sont admissibles, elles donneront toujours le même chemin

Voici une façon de trouver un tel cas:

In [10]:
for nameA, cityA in cityDict.items():
    for nameB, cityB in cityDict.items():
            pathCount=set()
            pathCount.add(len(findShortestPathFromAtoB(cityA, cityB, h4,False)))
            pathCount.add(len(findShortestPathFromAtoB(cityA, cityB, h3,False)))
            if len(pathCount)>1:
                print(f"{nameA} - {nameB}")


Amsterdam - Prague
Brussels - Prague
Paris - Prague


On peut constater que Brussels - Prague et Paris - Prague passera forcément par Amsterdam donc on peut se concentrer sur le cas Amsterdam - Prague:

In [11]:
%run tp2_AStar_Sergiy_Goloviatinski.py Amsterdam Prague

Find shorthest path from Amsterdam to Prague with 5 different heuristics:

	Heuristic h0 = lambda a,b: 0

	Found path (length = 3) from Amsterdam to Prague after exploring 8 states
		CityState: Amsterdam, g: 0 h:0 f:0
		 ⮡ CityState: Munich, g: 526 h:0 f:526
		  ⮡ CityState: Prague, g: 700 h:0 f:700

	Heuristic h1 = lambda a,b: abs(a.x-b.x)

	Found path (length = 3) from Amsterdam to Prague after exploring 6 states
		CityState: Amsterdam, g: 0 h:383 f:383
		 ⮡ CityState: Munich, g: 526 h:105 f:631
		  ⮡ CityState: Prague, g: 700 h:0 f:700

	Heuristic h2 = lambda a,b: abs(a.y-b.y)

	Found path (length = 3) from Amsterdam to Prague after exploring 6 states
		CityState: Amsterdam, g: 0 h:121 f:121
		 ⮡ CityState: Munich, g: 526 h:140 f:666
		  ⮡ CityState: Prague, g: 700 h:0 f:700

	Heuristic h3 = lambda a,b: sqrt(abs(a.x-b.x)**2 + abs(a.y-b.y)**2)

	Found path (length = 3) from Amsterdam to Prague after exploring 5 states
		CityState: Amsterdam, g: 0 h:401.6590594023742 f:401.65905940237

Dans l'exemple ci-dessus le chemin trouvé avec l'heuristique h4 passe par
- Amsterdam -> Hamburg -> Berlin -> Prague 

à la place de 
- Amsterdam -> Munich -> Prague 

comme toutes les autres heuristiques

Explications pour la différence de choix à partir d'Amsterdam pour aller à Prague: 

On analyse la valeur de l'heuristique h4, le coût réel et les distances en x et y entre Munich et Prague:

In [12]:
h4(cityDict["Munich"],cityDict["Prague"])

245

In [13]:
#distance Munich->Prague
cityDict["Munich"].neighbours[cityDict["Prague"]]

174

In [14]:
# distance en valeur absolue en x (h1) entre Munich et Prague
h1(cityDict["Munich"],cityDict["Prague"])

105

In [15]:
# distance en valeur absolue en y (h2) entre Munich et Prague
h2(cityDict["Munich"],cityDict["Prague"])

140

Et maintenent entre Hamburg et Prague:

In [16]:
h4(cityDict["Hamburg"],cityDict["Prague"])

400

In [17]:
#distance Hamburg->Prague en passant par Berlin
cityDict["Hamburg"].neighbours[cityDict["Berlin"]] + cityDict["Berlin"].neighbours[cityDict["Prague"]]

401

In [18]:
# distance en valeur absolue en x (h1) entre Munich et Prague
h1(cityDict["Hamburg"],cityDict["Prague"])

200

In [19]:
# distance en valeur absolue en y (h2) entre Munich et Prague
h2(cityDict["Hamburg"],cityDict["Prague"])

200

L'heuristique h4 surestime le coût entre Munich et Prague, ça aurait aussi été le cas entre Hamburg et Prague, d'autant plus que la différence en x et y entre ces deux villes est égale donc c'est la pire surestimation possible pour cette heuristique, sauf que il n'y a pas de connexion directe, donc vu qu'il faut passer par Berlin pour aller de Hamburg à Prague cette surestimation s'estompe avec l'addition du coût réel Berlin->Prague (c'était limite en plus vu que le coût réel est de 1 plus grand que l'estimation)

## Dans un cas réel, quelle heuristique utiliseriez-vous ?

In [20]:
%run tp2_AStar_Sergiy_Goloviatinski.py lisbon warsaw copenhagen naples

Find shorthest path from Lisbon to Warsaw with 5 different heuristics:

	Heuristic h0 = lambda a,b: 0

	Found path (length = 8) from Lisbon to Warsaw after exploring 19 states
		CityState: Lisbon, g: 0 h:0 f:0
		 ⮡ CityState: Madrid, g: 339 h:0 f:339
		  ⮡ CityState: Paris, g: 1144 h:0 f:1144
		   ⮡ CityState: Brussels, g: 1369 h:0 f:1369
		    ⮡ CityState: Amsterdam, g: 1533 h:0 f:1533
		     ⮡ CityState: Hamburg, g: 1871 h:0 f:1871
		      ⮡ CityState: Berlin, g: 2053 h:0 f:2053
		       ⮡ CityState: Warsaw, g: 2398 h:0 f:2398

	Heuristic h1 = lambda a,b: abs(a.x-b.x)

	Found path (length = 8) from Lisbon to Warsaw after exploring 18 states
		CityState: Lisbon, g: 0 h:1280 f:1280
		 ⮡ CityState: Madrid, g: 339 h:1036 f:1375
		  ⮡ CityState: Paris, g: 1144 h:723 f:1867
		   ⮡ CityState: Brussels, g: 1369 h:644 f:2013
		    ⮡ CityState: Amsterdam, g: 1533 h:618 f:2151
		     ⮡ CityState: Hamburg, g: 1871 h:435 f:2306
		      ⮡ CityState: Berlin, g: 2053 h:287 f:2340
		       ⮡ CityStat

Si on regarde que le nombre de noeuds explorés avant de trouver le chemin le plus court, on aurait tendance à vouloir utiliser h4 vu que c'est avec cette heuristique que ce nombre est le plus petit.

Cependant, comme montré dans la question précédente cette heuristique n'est pas admissible donc elle risque de ne pas toujours donner le chemin le plus court, donc j'aurais utilisé l'heuristique h3 car c'est celle qui trouve le plus vite après h4 le chemin le plus court.